**OFFICIAL FILE DELLA LDA SU STAN**

In [ ]:
import os
import numpy as np
from cmdstanpy import cmdstan_path, CmdStanModel

import arviz as az
import seaborn as sns
import matplotlib.pyplot as plt

from tensorflow_probability.substrates import numpy as tfp
tfd = tfp.distributions

In [ ]:
import cmdstanpy
cmdstanpy.install_cmdstan()
cmdstanpy.install_cmdstan(compiler=True)

Installing CmdStan version: 2.31.0
Install directory: /root/.cmdstan
Download successful, file: /tmp/tmpwh78_rvl
Extracting distribution


DEBUG:cmdstanpy:cmd: make build -j1
cwd: None


Unpacked download as cmdstan-2.31.0
Building version cmdstan-2.31.0, may take several minutes, depending on your system.


DEBUG:cmdstanpy:cmd: make examples/bernoulli/bernoulli
cwd: None


Test model compilation
Installed cmdstan-2.31.0
Installing CmdStan version: 2.31.0
Install directory: /root/.cmdstan
CmdStan version 2.31.0 already installed


True

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

Import the dataset: tf_measure

In [ ]:
data = pd.read_csv('tf.csv')

In [ ]:
data.head()

,Unnamed: 0,distanc,well-equip,concur,goos,left-field,matrix,neurotransmitt,talbott,equip,...,un-us,.becaus,bodywork,infatu,pratchett,awar,dude,jp,sector,type_text
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


Remove the first and last column:

*   the first is the row index
*   the last contains the labels



In [ ]:
dat = data.iloc[:,1:17094]
dat.shape

(2000, 17093)

In [ ]:
dat.head()

,distanc,well-equip,concur,goos,left-field,matrix,neurotransmitt,talbott,equip,relax,...,harmless,un-us,.becaus,bodywork,infatu,pratchett,awar,dude,jp,sector
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Create a list of words

In [ ]:
list_of_words = dat.columns.values.tolist()



*   Save the vector 'word istances' corresponding to each word, i.e. the number of times that a word appear in a document (num_words) 

*   Save the index of the words that appears more than TOT times (idx)

In [ ]:
num_words = dat.sum(axis = 0).to_numpy() 
TOT = 10
idx = np.asarray(np.where(num_words > TOT))[0,:]

Save in red_list_of_words the words that appear more than TOT times in the dataset

In [ ]:
red_list_of_words = [list_of_words[x] for x in idx]

In [ ]:
len(red_list_of_words) 

1840

Create a dictionary with the words that appear more than TOT times in the datset

In [ ]:
dictionary = dict(zip(np.array(range(1,len(red_list_of_words)+1)),red_list_of_words))
dictionary

{1: 'equip',
 2: 'steve',
 3: 'shallow',
 4: 'lyric',
 5: 'still',
 6: 'portray',
 7: 'hel',
 8: 'overview',
 9: 'experienc',
 10: '4',
 11: 'serv',
 12: 'cours',
 13: 'son',
 14: 'manag',
 15: 'found',
 16: 'folk',
 17: 'everi',
 18: 'done',
 19: 'balanc',
 20: 'food',
 21: '900',
 22: 'sister',
 23: 'hardli',
 24: 'remak',
 25: 'knowledg',
 26: 'worth',
 27: 'goe',
 28: 'complex',
 29: 'lie',
 30: 'journey',
 31: 'origin',
 32: 'innoc',
 33: 'public',
 34: 'eventu',
 35: 'slow',
 36: 'ie',
 37: 'stuff',
 38: 'consequ',
 39: 'punch',
 40: 'plan',
 41: 'patient',
 42: 'chanc',
 43: 'amateur',
 44: 'adult',
 45: 'doctor',
 46: 'fast',
 47: 'tend',
 48: 'bear',
 49: 'excus',
 50: 'term',
 51: 'artist',
 52: 'author',
 53: 'pictur',
 54: 'critic',
 55: 'paul',
 56: 'local',
 57: 'hero',
 58: 'vegetarian',
 59: 'comparison',
 60: 'crowley',
 61: 'prais',
 62: 'review',
 63: 'worker',
 64: 'confus',
 65: 'want',
 66: 'social',
 67: 'analyz',
 68: 'mass',
 69: 'suit',
 70: 'deep',
 71: 'bulk

**PREPROCESSING DEL DATASET**


*   Create vectors w and doc
*   If you want to reduce the dataset (remove words that appear a few times and reduce the number of documents make all cells go and you get the new reduced dataset : datum). If not instead of datum when creating w and doc use the full dataset : dat



Saving 100 positive reviews and 100 negative reviews.
To speed up the sampling.


Otherwise select all and use STAN optimize method

In [ ]:
pos = dat.iloc[120:320].to_numpy()
neg = dat.iloc[1001:1101].to_numpy()
pos.shape,neg.shape

((200, 17093), (100, 17093))

Create the reduced dataset, i.e. with less documents

In [ ]:
red_data = np.concatenate((pos,neg),axis = 0)
red_data.shape

(300, 17093)

Keep only the words that appear more than TOT times

In [ ]:
red_dat = red_data[:,idx]
red_dat.shape

(300, 1840)

In [ ]:
datum = pd.DataFrame(red_dat)
datum.head()

,0,1,2,3,4,5,6,7,8,9,...,1830,1831,1832,1833,1834,1835,1836,1837,1838,1839
0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


Save total_word_istances, i.e. the number of total words that appear in the dataset

In [ ]:
total_word_istances = datum.values.sum() 
total_word_istances

25832



*   Create the vector w, i.e. the vector that contains all the words in the dataset with their index in the vocabulary.



In [ ]:
w = np.array([0]) 

In [ ]:
for i in range(0, datum.shape[0]):
  row = np.where(datum.iloc[i,:] > 0) 
  for j in range(0,len(row[0])):
      temp = np.repeat(int(row[0][j])+1,datum.iloc[i,row[0][j]],axis = 0)  
      w = np.append(w,temp)

In [ ]:
w = w[1:] 
w.shape

(25832,)

Check that there are no zeros in w

In [ ]:
np.where(w == 0)

(array([], dtype=int64),)



*   Devo creare il vettore doc da dare alla lda, ovvero il vettore che ha come entrate a che documento appartiene ogni parola che entra nel dataset



In [ ]:
doc_len = datum.sum(axis = 1) 

In [ ]:
doc = np.array([0])

In [ ]:
for i in range(datum.shape[0]):
  temp = np.repeat(i+1,doc_len[i],axis = 0) 
  doc = np.append(doc,temp)


Check that doc and w have the same dimensions

In [ ]:
doc = doc[1:]
doc.shape

(25832,)



*  Setting priors and parameters



In [ ]:
K = 8 #number of topics
V = datum.shape[1] #number of unique words
M = datum.shape[0] #number of documents
N = doc.shape[0] # dimensionality of w and doc, i.e. total word instances

In [ ]:
alpha =  np.repeat(1, K) # prior on the topics

In [ ]:
beta = np.repeat(1, V) # prior on the words

Model Reference: [clicca_qui](https://mc-stan.org/docs/stan-users-guide/latent-dirichlet-allocation.html)

In [ ]:
lda2_code = """
data {
  int<lower=2> K;               // num topics
  int<lower=2> V;               // num words
  int<lower=1> M;               // num docs
  int<lower=1> N;               // total word instances
  array[N] int<lower=1, upper=V> w;    // word n
  array[N] int<lower=1, upper=M> doc;  // doc ID for word n
  vector<lower=0>[K] alpha;     // topic prior
  vector<lower=0>[V] beta;      // word prior
}
parameters {
  array[M] simplex[K] theta;    // topic dist for doc m
  array[K] simplex[V] phi;      // word dist for topic k
}
model {
  
  theta ~ dirichlet(alpha);  // prior

  phi ~ dirichlet(beta);     // prior
  
  for (n in 1:N) {
    array[K] real gamma;
    for (k in 1:K) {
      gamma[k] = log(theta[doc[n], k]) + log(phi[k, w[n]]);
    }
    target += log_sum_exp(gamma);  // likelihood;
  }
}
"""

stan_file2 = "./lda_model2.stan"

with open(stan_file2, "w") as f:
    print(lda2_code, file=f)

stan_model2 = CmdStanModel(stan_file=stan_file2)

21:29:33 - cmdstanpy - INFO - compiling stan file /content/lda_model2.stan to exe file /content/lda_model2
INFO:cmdstanpy:compiling stan file /content/lda_model2.stan to exe file /content/lda_model2
DEBUG:cmdstanpy:cmd: make /content/lda_model2
cwd: /root/.cmdstan/cmdstan-2.31.0
DEBUG:cmdstanpy:Console output:

--- Translating Stan model to C++ code ---
bin/stanc  --o=/content/lda_model2.hpp /content/lda_model2.stan

--- Compiling, linking C++ code ---
g++ -std=c++1y -pthread -D_REENTRANT -Wno-sign-compare -Wno-ignored-attributes      -I stan/lib/stan_math/lib/tbb_2020.3/include    -O3 -I src -I stan/src -I stan/lib/rapidjson_1.1.0/ -I lib/CLI11-1.9.1/ -I stan/lib/stan_math/ -I stan/lib/stan_math/lib/eigen_3.3.9 -I stan/lib/stan_math/lib/boost_1.78.0 -I stan/lib/stan_math/lib/sundials_6.1.1/include -I stan/lib/stan_math/lib/sundials_6.1.1/src/sundials    -DBOOST_DISABLE_ASSERTS          -c -Wno-ignored-attributes   -x c++ -o /content/lda_model2.o /content/lda_model2.hpp
g++ -std=c++1y 

In [ ]:
normal_data = {
    "K": K,
    "V": V,
    "M": M,
    "N": N,
    "w": w,
    "doc": doc,
    "alpha": alpha,
    "beta": beta
}

stan_fit = stan_model2.sample(data=normal_data, chains=4, 
                             parallel_chains=4, 
                             iter_warmup=400, iter_sampling=2000) #otherwise use optimize method

cmdstanpy_data = az.from_cmdstanpy(stan_fit)

Output analysis

In [ ]:
stan_fit.summary()

DEBUG:cmdstanpy:cmd: /root/.cmdstan/cmdstan-2.31.0/bin/stansummary --percentiles= 5,50,95 --sig_figs=6 --csv_filename=/tmp/tmpf83w9hdd/stansummary-lda_model2-itbaap6n.csv /tmp/tmpf83w9hdd/lda_model2s296rnoo/lda_model2-20221218160913_1.csv /tmp/tmpf83w9hdd/lda_model2s296rnoo/lda_model2-20221218160913_2.csv /tmp/tmpf83w9hdd/lda_model2s296rnoo/lda_model2-20221218160913_3.csv /tmp/tmpf83w9hdd/lda_model2s296rnoo/lda_model2-20221218160913_4.csv
cwd: None


,Mean,MCSE,StdDev,5%,50%,95%,N_Eff,N_Eff/s,R_hat
lp__,-411040.000000,2.674270,88.503000,-411186.000000,-411040.000000,-410896.000000,1095.23000,0.094867,1.000830
"theta[1,1]",0.140261,0.000826,0.057064,0.058081,0.133952,0.242412,4768.97000,0.413080,1.000670
"theta[1,2]",0.213697,0.089470,0.140282,0.064904,0.162005,0.503321,2.45836,0.000213,2.217740
"theta[1,3]",0.216895,0.092352,0.144150,0.065054,0.161786,0.511077,2.43636,0.000211,2.269620
"theta[1,4]",0.286611,0.105754,0.164121,0.070134,0.268008,0.546264,2.40844,0.000209,2.325660
...,...,...,...,...,...,...,...,...,...
"phi[5,2497]",0.000546,0.000005,0.000306,0.000149,0.000487,0.001115,3885.11000,0.336522,0.999298
"phi[5,2498]",0.000394,0.000003,0.000223,0.000110,0.000350,0.000810,5346.26000,0.463085,0.999594
"phi[5,2499]",0.000359,0.000003,0.000207,0.000095,0.000323,0.000748,5251.47000,0.454874,0.999123
"phi[5,2500]",0.000411,0.000003,0.000235,0.000112,0.000368,0.000854,4568.99000,0.395759,1.000160


In [ ]:
summa = pd.DataFrame(stan_fit.summary())

DEBUG:cmdstanpy:cmd: /root/.cmdstan/cmdstan-2.31.0/bin/stansummary --percentiles= 5,50,95 --sig_figs=6 --csv_filename=/tmp/tmpf83w9hdd/stansummary-lda_model2-16rm3fwm.csv /tmp/tmpf83w9hdd/lda_model2s296rnoo/lda_model2-20221218160913_1.csv /tmp/tmpf83w9hdd/lda_model2s296rnoo/lda_model2-20221218160913_2.csv /tmp/tmpf83w9hdd/lda_model2s296rnoo/lda_model2-20221218160913_3.csv /tmp/tmpf83w9hdd/lda_model2s296rnoo/lda_model2-20221218160913_4.csv
cwd: None


In [ ]:
summa.head()

,Mean,MCSE,StdDev,5%,50%,95%,N_Eff,N_Eff/s,R_hat
lp__,-411040.000000,2.674270,88.503000,-411186.000000,-411040.000000,-410896.000000,1095.23000,0.094867,1.00083
"theta[1,1]",0.140261,0.000826,0.057064,0.058081,0.133952,0.242412,4768.97000,0.413080,1.00067
"theta[1,2]",0.213697,0.089470,0.140282,0.064904,0.162005,0.503321,2.45836,0.000213,2.21774
"theta[1,3]",0.216895,0.092352,0.144150,0.065054,0.161786,0.511077,2.43636,0.000211,2.26962
"theta[1,4]",0.286611,0.105754,0.164121,0.070134,0.268008,0.546264,2.40844,0.000209,2.32566
